In [32]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from xml.etree import ElementTree as ET
from statsmodels.stats.multitest import multipletests

In [33]:
df=pd.read_csv("../../csv2_new_05_11_without_correct_pvalue.csv")
df

,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description,Average of P,Std of P,P-value
0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,immunoglobulin kappa variable 2D-40 [Source:HG...,0.032301,0.110641,8.145800e-10
1,1,ACB,0,chr5:676694-846549,ENSG00000289088,NaN,chr5:693493-694831,F,novel transcript,0.137223,0.328793,0.000000e+00
2,2,ACB,0,chr5:676694-846549,ENSG00000206077,ZDHHC11B,chr5:710355-784729,R,zinc finger DHHC-type containing 11B [Source:H...,0.011208,0.143739,5.113493e-04
3,3,ACB,0,chr5:676694-846549,ENSG00000249908,BRD9P2,chr5:767382-768930,R,bromodomain containing 9 pseudogene 2 [Source:...,0.059533,0.160758,0.000000e+00
4,4,ACB,0,chr5:676694-846549,ENSG00000285522,NaN,chr5:770005-770113,R,tubulin polymerization promoting protein (TPPP...,0.032256,0.175496,8.445400e-10
...,...,...,...,...,...,...,...,...,...,...,...,...
1684,1684,YRI,1,chr6:32589647-32882258,ENSG00000204264,PSMB8,chr6:32840717-32844679,R,proteasome 20S subunit beta 8 [Source:HGNC Sym...,-0.020310,0.068947,8.334643e-01
1685,1685,YRI,1,chr6:32589647-32882258,ENSG00000204261,PSMB8-AS1,chr6:32844078-32846500,F,PSMB8 antisense RNA 1 (head to head) [Source:H...,-0.013347,0.049097,5.193934e-01
1686,1686,YRI,1,chr6:32589647-32882258,ENSG00000240065,PSMB9,chr6:32844136-32859851,F,proteasome 20S subunit beta 9 [Source:HGNC Sym...,-0.016676,0.077383,6.872762e-01
1687,1687,YRI,1,chr6:32589647-32882258,ENSG00000168394,TAP1,chr6:32845209-32853816,R,"transporter 1, ATP binding cassette subfamily ...",-0.029619,0.059606,9.859862e-01


In [34]:
df_floats = df[df['Position of gene'].apply(lambda x: isinstance(x, float))]

In [35]:
df_floats

,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description,Average of P,Std of P,P-value
98,98,BEB,0,chr2:56405225-56644106,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
116,116,BEB,0,chr4:115247002-115485969,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
295,295,CHB,0,chr4:115247002-115485969,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
316,316,CHB,0,chr22:16175432-16345129,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
414,414,CLM,0,chr12:61013177-61213757,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
557,557,GBR,0,chr2:56405225-56644106,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
848,848,JPT,0,chr22:16175432-16345129,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
967,967,MXL,0,chr5:11999758-12263260,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
991,991,MXL,0,chr12:73338587-73586502,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN
1121,1121,PJL,0,chr17:52575391-52861780,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN


In [36]:
df_without_floats = df.drop(df_floats.index)
df_without_floats

,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description,Average of P,Std of P,P-value
0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,immunoglobulin kappa variable 2D-40 [Source:HG...,0.032301,0.110641,8.145800e-10
1,1,ACB,0,chr5:676694-846549,ENSG00000289088,NaN,chr5:693493-694831,F,novel transcript,0.137223,0.328793,0.000000e+00
2,2,ACB,0,chr5:676694-846549,ENSG00000206077,ZDHHC11B,chr5:710355-784729,R,zinc finger DHHC-type containing 11B [Source:H...,0.011208,0.143739,5.113493e-04
3,3,ACB,0,chr5:676694-846549,ENSG00000249908,BRD9P2,chr5:767382-768930,R,bromodomain containing 9 pseudogene 2 [Source:...,0.059533,0.160758,0.000000e+00
4,4,ACB,0,chr5:676694-846549,ENSG00000285522,NaN,chr5:770005-770113,R,tubulin polymerization promoting protein (TPPP...,0.032256,0.175496,8.445400e-10
...,...,...,...,...,...,...,...,...,...,...,...,...
1684,1684,YRI,1,chr6:32589647-32882258,ENSG00000204264,PSMB8,chr6:32840717-32844679,R,proteasome 20S subunit beta 8 [Source:HGNC Sym...,-0.020310,0.068947,8.334643e-01
1685,1685,YRI,1,chr6:32589647-32882258,ENSG00000204261,PSMB8-AS1,chr6:32844078-32846500,F,PSMB8 antisense RNA 1 (head to head) [Source:H...,-0.013347,0.049097,5.193934e-01
1686,1686,YRI,1,chr6:32589647-32882258,ENSG00000240065,PSMB9,chr6:32844136-32859851,F,proteasome 20S subunit beta 9 [Source:HGNC Sym...,-0.016676,0.077383,6.872762e-01
1687,1687,YRI,1,chr6:32589647-32882258,ENSG00000168394,TAP1,chr6:32845209-32853816,R,"transporter 1, ATP binding cassette subfamily ...",-0.029619,0.059606,9.859862e-01


In [37]:
pvalue_list=df_without_floats["P-value"].tolist()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
dfnew = df_without_floats.assign(corr_pvals=corr_pvals)

In [38]:
dfnew

,Unnamed: 0,Race,isMHC,Position of disassortative mating region,Gene ID,Gene name,Position of gene,is_complement,Gene description,Average of P,Std of P,P-value,corr_pvals
0,0,ACB,0,chr2:89762579-89852967,ENSG00000251039,IGKV2D-40,chr2:89851791-89852493,F,immunoglobulin kappa variable 2D-40 [Source:HG...,0.032301,0.110641,8.145800e-10,3.538991e-09
1,1,ACB,0,chr5:676694-846549,ENSG00000289088,NaN,chr5:693493-694831,F,novel transcript,0.137223,0.328793,0.000000e+00,0.000000e+00
2,2,ACB,0,chr5:676694-846549,ENSG00000206077,ZDHHC11B,chr5:710355-784729,R,zinc finger DHHC-type containing 11B [Source:H...,0.011208,0.143739,5.113493e-04,1.147969e-03
3,3,ACB,0,chr5:676694-846549,ENSG00000249908,BRD9P2,chr5:767382-768930,R,bromodomain containing 9 pseudogene 2 [Source:...,0.059533,0.160758,0.000000e+00,0.000000e+00
4,4,ACB,0,chr5:676694-846549,ENSG00000285522,NaN,chr5:770005-770113,R,tubulin polymerization promoting protein (TPPP...,0.032256,0.175496,8.445400e-10,3.659673e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,1684,YRI,1,chr6:32589647-32882258,ENSG00000204264,PSMB8,chr6:32840717-32844679,R,proteasome 20S subunit beta 8 [Source:HGNC Sym...,-0.020310,0.068947,8.334643e-01,8.896878e-01
1685,1685,YRI,1,chr6:32589647-32882258,ENSG00000204261,PSMB8-AS1,chr6:32844078-32846500,F,PSMB8 antisense RNA 1 (head to head) [Source:H...,-0.013347,0.049097,5.193934e-01,5.978193e-01
1686,1686,YRI,1,chr6:32589647-32882258,ENSG00000240065,PSMB9,chr6:32844136-32859851,F,proteasome 20S subunit beta 9 [Source:HGNC Sym...,-0.016676,0.077383,6.872762e-01,7.602652e-01
1687,1687,YRI,1,chr6:32589647-32882258,ENSG00000168394,TAP1,chr6:32845209-32853816,R,"transporter 1, ATP binding cassette subfamily ...",-0.029619,0.059606,9.859862e-01,1.000000e+00


In [40]:
dfnew.to_csv("csv2withcorr_withoutfloat.csv")

In [42]:
df_floats.to_csv("csv2_df_floats.csv")